# Model Evaluation

In [ ]:
from glob import glob
from sklearn.externals.joblib import load
from keras.models import load_model
from ipywidgets import interact, widgets
from IPython.display import display
from pulsar_playground.plots import *

In [ ]:
models = glob('./saved_models/*.joblib')
models.extend(glob('./saved_models/*.h5'))

In [ ]:
test = pd.read_csv('./pulsar_playground/dataset/test_set.csv')
X_test = test.iloc[:,:-1]
y_test = test.iloc[:, -1]

In [ ]:
@interact(model=models, threshold='0.5', save_candidates=False)
def model_eval(model, threshold=0.5, save_candidates=False):
        
    try:
        threshold = float(threshold)
        
    except ValueError:
        threshold = 0.
    
    
    if model[-2:] == 'h5':        
        clf = load_model(model)
        y_positive_class = clf.predict_proba(X_test)
        y_negative_class = 1 - y_positive_class
        
        # KerasClassifier predict_proba output has only one row.       
        y_pred_proba = np.hstack((y_negative_class, y_positive_class))
    
    else:        
        clf = load(model)
        y_pred_proba = clf.predict_proba(X_test)
    
        
    fig1, (ax1, ax2) = plt.subplots(figsize=(12,6), nrows=1, ncols=2)
    
    plot_prc(y_test, y_pred_proba, threshold, ax=ax1)
    plot_cm(y_test, y_pred_proba, threshold, ax=ax2)
    
    
    if save_candidates:
        dump_idx(y_pred_proba, threshold)